In [1]:
import os
import cv2
import json
import matplotlib.pyplot as plt
import numpy as np
from pycococreatortools import pycococreatortools
import pprint

home_path=os.path.expanduser('~')
mask_dir=os.path.join(home_path, 'weapons/googleguns/annotations/consolidated-annotation/output')
json_ann_file=os.path.join(home_path, '/home/m/weapons/googleguns/intermediate/1/output.manifest')

In [2]:
def get_binary_mask(img, color):
    lower_color = np.array(color)-3
    upper_color = np.array(color)
    binary_mask = cv2.inRange(img, lower_color, upper_color)
    return binary_mask

In [3]:
def get_contours(img, color):
    binary_mask = get_binary_mask(img, color)
    contours, hierarchy = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if contours:
        img_con = cv2.drawContours(img, contours, -1, (0, 255, 0), 3)
        cv2.imshow("test", img_con)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    return contours

In [4]:
def get_img(s3_path):
    splitted = s3_path.split('/')
    img_local_path = '/'.join(splitted[2:])
    img_full_path = os.path.join(home_path, img_local_path)
    img = cv2.imread(img_full_path)
    return img
    

In [5]:
def get_cls_annots(cats_ids):
    cls = []
    cls.append({"supercategory": "gun","name": "rifle", "id": cats_ids['rifle']})
    cls.append({"supercategory": "gun","name": "pistol", "id": cats_ids['pistol']})
    cls.append({"supercategory": "gun","name": "person", "id": cats_ids['person']})
    return cls 

def get_image_annot(im_id, width, height, filename):
    image_annotation = {}
    image_annotation['id'] = im_id
    image_annotation['width'] = width
    image_annotation['height'] = height
    image_annotation['license'] = 0
    image_annotation["file_name"] = filename
    return image_annotation

def get_annot(segmentation, cat_id, im_id, ann_id_begin):
    annotations = []
    for i, segment in enumerate(segmentation):
        annotation = {}
        x, y, width, height = get_bnd_box(segment)
        annotation['segmentation'] = segment
        annotation['area'] = width*height
        annotation['is_crowd'] = 0
        annotation['image_id'] = im_id
        annotation['bbox'] = [x,y, width, height]
        annotation['category_id'] = cat_id
        annotation['id'] = ann_id_begin + i
        annotations.append(annotation)
    return annotations

In [6]:
def get_min(actual, prev_min):
    return actual if actual < prev_min else prev_min

def get_max(actual, prev_max):
    return actual if actual > prev_max else prev_max

def get_bnd_box(segment):
    min_x, min_y, max_x, max_y = 10000,10000,0,0
    for i in range(0, len(segment),2):
        x, y = segment[i:i+2]
        
        min_x, min_y = get_min(x, min_x), get_min(y, min_y)
        max_x, max_y = get_max(x, max_x), get_max(y, max_y)
    
    width, height = max_x - min_x, max_y - min_y
    return min_x, min_y, width, height
    

In [7]:
def get_cls_ids(in_dict):
    cls_ids = {}
    if "googleguns-ref-metadata" not in in_dict:
        return None
    in_dict_roi = in_dict["googleguns-ref-metadata"]["internal-color-map"]
    for key in in_dict_roi.keys():
        cls_ids[in_dict_roi[key]["class-name"]] = key
    
    return cls_ids

In [8]:
annotations = []
images = []
with open(json_ann_file, 'r') as f:
    lines = f.readlines() 
    for x, line in enumerate(lines):
        json_data = json.loads(line)
        im_id = x
        cls_ids = get_cls_ids(json_data) if get_cls_ids(json_data) else cls_ids
        if 'googleguns-ref' not in json_data:
            continue
        s3_ann_img_path = json_data['googleguns-ref']
        s3_src_img_path_tail = os.path.split(json_data['source-ref'])[1]
        for i in range(1,4):
            info = json_data['googleguns-ref-metadata']['internal-color-map'][str(i)]
            color_info = info['hex-color'][1:]
            int_color_info = int(color_info, 16) 
            r_col =  (int_color_info & 0xff0000) >> 16
            g_col = (int_color_info & 0xff00) >> 8
            b_col = int_color_info & 0xff
            color = [b_col, g_col, r_col]
            ann_img = get_img(s3_ann_img_path)
            b = get_binary_mask(ann_img, color)
            annotation_info = pycococreatortools.create_annotation_info(
                    1, 1, {'is_crowd':0, "id":i},  b,
                    ann_img.shape[:2], tolerance=2)
            if annotation_info:
                anns = get_annot(annotation_info['segmentation'],cat_id=i, im_id=im_id, ann_id_begin= im_id*10)
        imgs_anns = get_image_annot(im_id, ann_img.shape[0], ann_img.shape[1], s3_src_img_path_tail)  
        annotations.append(anns) 
        images.append(imgs_anns)
categories  = get_cls_annots(cls_ids)

In [9]:
end_json = {}
end_json['type'] = "instances"
end_json['images'] = images
end_json['categories'] = categories
end_json["annotations"] = annotations

In [10]:
annotation_file='/home/m/glock/annotations_glock/test.json'
os.makedirs(os.path.split(annotation_file)[0], exist_ok=True)
with open(annotation_file, 'w') as f:
    json.dump(end_json, f)